# IEEEP370 Deembedding

## Target

The aim of this example is to illustrate the use of IEEEP370 deembedding methods.

In [ ]:
%load_ext autoreload
%autoreload 2
import skrf as rf
import matplotlib.pyplot as plt
from skrf.calibration import Ieeep370nzc2xthru
from skrf.calibration import Ieeep370zc2xthru
from skrf.media import MLine
rf.stylely()

## Simulation of DUT and Fixtures
Inspired by [this example](https://scikit-rf.readthedocs.io/en/latest/examples/networktheory/Time%20domain%20reflectometry%2C%20measurement%20vs%20simulation.html).

In [ ]:
freq = rf.F(1e-3,10,10000, 'ghz')
W   = 3.20e-3 #3.00e-3 for 50 ohm
H   = 1.51e-3
T   = 50e-6
ep_r = 4.413
tanD = 0.0182
f_epr_tand = 1e9
# microstrip segments
MSL1 = MLine(frequency=freq, z0=50, w=W, h=H, t=T,
        ep_r=ep_r, mu_r=1, rho=1.712e-8, tand=tanD, rough=0.15e-6,
        f_low=1e3, f_high=1e12, f_epr_tand=f_epr_tand,
        diel='djordjevicsvensson', disp='kirschningjansen')
# capacitive 3 x width Beatty structure
MSL2 = MLine(frequency=freq, z0=50, w=3*W, h=H, t=T,
        ep_r=ep_r, mu_r=1, rho=1.712e-8, tand=tanD, rough=0.15e-6,
        f_low=1e3, f_high=1e12, f_epr_tand=f_epr_tand,
        diel='djordjevicsvensson', disp='kirschningjansen')
# building DUT
dut =    MSL1.line(20e-3, 'm', embed=True, z0=MSL1.Z0_f) \
      ** MSL2.line(20e-3, 'm', embed=True, z0=MSL2.Z0_f) \
      ** MSL1.line(20e-3, 'm', embed=True, z0=MSL1.Z0_f)
dut.name = 'dut'
# building FIXTURE-DUT-FIXTURE
thru = MSL1.line(30e-3, 'm', embed=True, z0=MSL1.Z0_f)
fdf     = thru ** dut ** thru
fdf.name = 'fdf'
# building FIXTURE-FIXTURE (2xthru)
s2xthru = thru ** thru
s2xthru.name = 's2xthru'

## Looking at generated networks
There is a need to extrapolate DC point to plot time step data.

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Time Step')
dut_dc = dut.extrapolate_to_dc(kind='cubic')
fdf_dc = fdf.extrapolate_to_dc(kind='cubic')
s2xthru_dc = s2xthru.extrapolate_to_dc(kind='cubic')
dut_dc.s11.plot_z_time_step(window='hamming')
fdf_dc.s11.plot_z_time_step(window='hamming')
s2xthru_dc.s11.plot_z_time_step(window='hamming')
plt.xlim((-2, 2))
plt.ylim((15, 55))
plt.legend(loc = 'lower left')
plt.subplot(1, 2, 2)
plt.title('Frequency')
dut.s11.plot_s_db()
fdf.s11.plot_s_db()
s2xthru.s11.plot_s_db()

## IEEEP370 nzc2xthru (no impedance correction)

In [ ]:
dm_nzc = Ieeep370nzc2xthru(dummy_2xthru = s2xthru, name = '2xthru')
nzc_side1 = dm_nzc.s_side1
nzc_side1.name = 'nzc_side1'
nzc_side2 = dm_nzc.s_side2
nzc_side2.name = 'nzc_side2'
nzc_d_dut = dm_nzc.deembed(fdf)
nzc_d_dut.name = 'nzc_d_dut'
# plot them all
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Time Step')
nzc_d_dut_dc = nzc_d_dut.extrapolate_to_dc(kind='cubic')
dut_dc.s11.plot_z_time_step(window='hamming')
nzc_d_dut_dc.s11.plot_z_time_step(window='hamming')
fdf_dc.s11.plot_z_time_step(window='hamming')
plt.xlim((-2, 2))
plt.ylim((15, 55))
plt.legend(loc = 'lower left')
plt.subplot(1, 2, 2)
plt.title('Frequency')
dut.plot_s_db(0, 0)
nzc_d_dut.plot_s_db(0, 0)
fdf.plot_s_db(0, 0)
dut.plot_s_db(1, 0)
nzc_d_dut.plot_s_db(1, 0)
fdf.plot_s_db(1, 0)
plt.ylim((-40, 5))

The NZC deembedding has removed the delay of the fixture but the impedance at t=0ns is wrong, causing an offset.

## IEEEP370 zc2xthru (impedance correction)

In [ ]:
dm_zc  = Ieeep370zc2xthru(dummy_2xthru = s2xthru, dummy_fix_dut_fix = fdf,
                         bandwidth_limit = 10e9, pullback1 = 0, pullback2 = 0,
                         leadin = 1,
                         name = 'zc2xthru')
zc_d_dut = dm_zc.deembed(fdf)
zc_d_dut.name = 'zc_d_dut'
zc_side1 = dm_zc.s_side1
zc_side1.name = 'nzc_side1'
zc_side2 = dm_zc.s_side2
zc_side2.name = 'nzc_side2'
zc_d_dut = dm_zc.deembed(fdf)
zc_d_dut.name = 'zc_d_dut'
# plot them all
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Time Step')
zc_d_dut_dc = zc_d_dut.extrapolate_to_dc(kind='cubic')
dut_dc.s11.plot_z_time_step(window='hamming')
zc_d_dut_dc.s11.plot_z_time_step(window='hamming')
fdf_dc.s11.plot_z_time_step(window='hamming')
plt.xlim((-2, 2))
plt.ylim((15, 55))
plt.legend(loc = 'lower left')
plt.subplot(1, 2, 2)
plt.title('Frequency')
dut.plot_s_db(0, 0)
zc_d_dut.plot_s_db(0, 0)
fdf.plot_s_db(0, 0)
dut.plot_s_db(1, 0)
zc_d_dut.plot_s_db(1, 0)
fdf.plot_s_db(1, 0)
plt.ylim((-40, 5))

The ZC deembedding show a better agreeement on time step and in the lower frequency domain.

## TODO
IEEEP370 recommand following consistency checks:
- Self de-embedding of 2x-thru with absolute magnitude of residual insertion loss < 0.1 dB and phase < 1 degree
- Compare the TDR of the fixture model to the FIX-DUT-FIX


In [ ]:
test1_nzc = nzc_side1.inv ** s2xthru ** nzc_side2.inv
test1_nzc.name = 'nzc'
test1_zc  = zc_side1.inv ** s2xthru ** zc_side2.inv
test1_zc.name = 'zc'
plt.figure(figsize=(10,5))
plt.subplot(2, 1, 1)
test1_nzc.plot_s_db(1,0)
test1_zc.plot_s_db(1,0)
plt.ylim((-0.1, 0.1))
plt.subplot(2, 1, 2)
test1_nzc.plot_s_deg(1,0)
test1_zc.plot_s_deg(1,0)
#plt.ylim((-1, 1))

The ZC deembedding, while giving a better agreement in time domain does not pass the consistency check. It will be interesting to compare with IEEE reference implementation results.